In [1]:
%pip install git+https://gitlab.com/moskalenkoviktor/ml_validation -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import locale

import numpy as np

from ml_validation import database
from ml_validation.experiment import mit_bih

In [3]:
locale.setlocale(locale.LC_ALL, "")
path_dataset = "datasets"

In [4]:
# Скачиваем базу MIT-BIH Atrial Fibrillation
database.download(database.Type.MIT_BIH_AF, path_dataset)

Archive already exists: datasets/mit_bih_af.zip


In [5]:
# Начинаем эксперимент
experiment = mit_bih.start_experiment(name="Random", authors="Moskalenko Viktor", path_dir=path_dataset)

In [6]:
class MyAlgorithm:
    # Алгоритм диагностики

    def __init__(self, rs: np.random.RandomState) -> None:
        self._rs = rs

    def __call__(self, dataset: mit_bih.TestDataset) -> list[mit_bih.YType]:
        return [self._rs.rand(len(indexes)) > 0.5 for _, indexes in dataset]

In [7]:
class MyTrainer:
    # Алгоритм обучения

    def __init__(self) -> None:
        self._fold = 0
        self._rs = np.random.RandomState(42)

    def __call__(self, dataset: mit_bih.TrainDataset) -> mit_bih.Algorithm:
        self._fold += 1
        print(f"Разбиение №{self._fold}. Число обучающих примеров: {len(dataset)}")
        return MyAlgorithm(self._rs)

In [8]:
# Валидируем с помощью 10-кратной перекрестной проверки
report = experiment.validate(MyTrainer())

Разбиение №1. Число обучающих примеров: 21
Разбиение №2. Число обучающих примеров: 21
Разбиение №3. Число обучающих примеров: 21
Разбиение №4. Число обучающих примеров: 20
Разбиение №5. Число обучающих примеров: 20
Разбиение №6. Число обучающих примеров: 20
Разбиение №7. Число обучающих примеров: 21
Разбиение №8. Число обучающих примеров: 21
Разбиение №9. Число обучающих примеров: 21
Разбиение №10. Число обучающих примеров: 21


In [9]:
print(report)

Версия: 0.0.12
Начало: Пн 17 июн 2024 19:15:26 MSK
Конец: Пн 17 июн 2024 19:15:32 MSK
Название: Random
Описание: 
Ссылка: 
Автор: Moskalenko Viktor

Метрики:
              NORM  NOT_NORM  accuracy  macro avg  weighted avg
precision 0.538625  0.460895  0.499736    0.49976      0.502781
recall    0.499472  0.500045  0.499736   0.499759      0.499736
f1-score   0.51831  0.479673  0.499736   0.498991      0.500493
support   608142.0  520419.0  0.499736  1128561.0     1128561.0

Матрицы рассогласования:

+---------------+---------------+
| NORM          | NOT_NORM      |
+===============+===============+
| 260233 260186 | 303750 304392 |
| 304392 303750 | 260186 260233 |
+---------------+---------------+

